In [ ]:
from fake_useragent import UserAgent 
from bs4 import BeautifulSoup    
import requests  
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm




# Функция по извлечению данных со страницы объявления в словарь data_dict
def parsing_page_one_ad(url):

    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})    
    response.encoding ='utf8'   
    
    # Теперь создадим объект BeautifulSoup, указывая html парсер    
    page = BeautifulSoup(response.text, 'html.parser')
    data_dict = {}

    data_dict['car_url'] = url
    data_dict['parsing_unixtime'] = int(time.time())


     # в разделах script ищем вхождение 'complectation":{"id"'
    for script in page.find_all("script"):
        if 'complectation":{"id"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
    # в a  ищем 'complectation":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]

        if 'equipment":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]

        if '{"mileage":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]

        if '"model_info":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')

        if 'super_gen":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:] 

        if 'vendor":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')


    for tag in page.find_all('div'):
        if tag.get("title") == "Идентификатор объявления":
            data_dict['sell_id'] = re.search(r'\d+', tag.text)[0]


    for tag in page.find_all("meta"):
        if tag.get("itemprop") == "bodyType":
            data_dict['bodyType'] = tag.get("content")

        if tag.get("itemprop") == "brand":
            data_dict['brand'] = tag.get("content")

        if tag.get("itemprop") == "color":
            data_dict['color'] = tag.get("content")

        if tag.get("itemprop") == "description":
            data_dict['description'] = tag.get("content")

        if tag.get("itemprop") == "engineDisplacement":
            data_dict['engineDisplacement'] = tag.get("content")

        if tag.get("itemprop") == "enginePower":
            data_dict['enginePower'] = tag.get("content")

        if tag.get("itemprop") == "fuelType":
            data_dict['fuelType'] = tag.get("content")

        if tag.get("itemprop") == "modelDate":
            data_dict['modelDate'] = tag.get("content")

        if tag.get("itemprop") == "name":
            data_dict['name'] = tag.get("content")

        if tag.get("itemprop") == "numberOfDoors":
            data_dict['numberOfDoors'] = tag.get("content")

        if tag.get("itemprop") == "price":
            data_dict['price'] = tag.get("content")

        if tag.get("itemprop") == "priceCurrency":
            data_dict['priceCurrency'] = tag.get("content")

        if tag.get("itemprop") == "productionDate":
            data_dict['productionDate'] = tag.get("content")

        if tag.get("itemprop") == "vehicleConfiguration":
            data_dict['vehicleConfiguration'] = tag.get("content")

        if tag.get("itemprop") == "vehicleTransmission":
            data_dict['vehicleTransmission'] = tag.get("content")


    span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})

    for i,tag in enumerate (span_CardInfoRow__cell):
        if tag.text == "Владельцы":
            data_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Владение":
            data_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "ПТС":
            data_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "Привод":
            data_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Руль":
            data_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Состояние":
            data_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Таможня":
            data_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

    return data_dict

    
# Функция по созданию списка ссылок links_list  на объявления о продаже автомобилей  
def extraction_links(url):
    links_list =[] 
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})  
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser') 
    links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for link in links:
        links_list.append(link.get("href"))
    return links_list
    
    
    
    
    
df = pd.DataFrame() # инициализируем итоговый датафрейм 

url_link_list = [] # список страниц по годам и номерам от 1 до 99

ranges = [range(1981, 2006), range(1, 100)] # указываем года и диапазон страниц, которые будем парсить 

# index[0] - year, index[1] - page 
for index in itertools.product(*ranges):
    # формируем ссылки страницы со списками объявлений
    url_links  = (f"https://auto.ru/moskva/cars/{index[0]}-year/all/?output_type=table&page={index[1]}")
    url_link_list.append(url_links) # заносим их в список
    
links_list = [] # список списков ссылок на объявления c одной страницы таблицы объявлений

#for url_links in url_link_list:
# извлекаем в links_list список ссылок на объявления
try:
        #links_list = extraction_links(url_links)
    links_list = Parallel(n_jobs = 2)(delayed(extraction_links)(url_links) for url_links in url_link_list)
except:
    pass 

ads_dict_list = [] # список словарей содержимого объявлений

for links in links_list:
    try:
        ads_dict_list = Parallel(n_jobs = 2)(delayed(parsing_page_one_ad)(ad_url) for ad_url in links)
    except:
        pass
    
    for ad in ads_dict_list:
        try:
            df = df.append(ad, ignore_index=True)
        except:
            pass

df.to_csv('_auto_ru_2081-2006.csv', encoding = 'utf-8', index=False) # записываем содержимое датафрейма в файл